# Project

### Step 2. 데이터 읽어오기

### glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도


In [34]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 0
Examples:
 []


## Step 3. 데이터 정제

### preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가면 잘라내기를 권

In [35]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [ ]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

print(tensor[:3, :10])

### 텐서 데이터는 모두 정수로 이루어져 있습니다. 이 숫자는 다름 아니라, tokenizer에 구축된 단어 사전의 인덱스입니다. 단어 사전이 어떻게 구축되었는지 아래와 같이 확인해 봅시다.

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 15: break

 ### 2번 인덱스가 바로 였습니다. 왜 모든 행이 2로 시작하는지 이해할 수 있겠습니다.





## Step 4. 평가 데이터셋 분리

### 훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!



In [51]:
enc_train, enc_val, dec_train, dec_val = train_test_split
(src_input, tgt_input, test_size=0.2)

SyntaxError: invalid syntax (<ipython-input-51-ed40bbd8e394>, line 2)

In [52]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

train_test_split(t)

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


NameError: name 'train_test_split' is not defined

### step 5. 인공지능 만들기

In [53]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [54]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)
src_sample.shape

TensorShape([256, 20])

In [55]:
model.summary()

Model: "text_generator_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      multiple                  3584512   
_________________________________________________________________
lstm_10 (LSTM)               multiple                  20979712  
_________________________________________________________________
lstm_11 (LSTM)               multiple                  33562624  
_________________________________________________________________
dense_5 (Dense)              multiple                  14345049  
Total params: 72,471,897
Trainable params: 72,471,897
Non-trainable params: 0
_________________________________________________________________


In [48]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
93/93 [==============================] - 630s 7s/step - loss: 3.3338
Epoch 2/10
93/93 [==============================] - 629s 7s/step - loss: 2.7044
Epoch 3/10
93/93 [==============================] - 630s 7s/step - loss: 2.5396
Epoch 4/10
93/93 [==============================] - 627s 7s/step - loss: 2.4297
Epoch 5/10
93/93 [==============================] - 636s 7s/step - loss: 2.3355
Epoch 6/10
93/93 [==============================] - 656s 7s/step - loss: 2.2438
Epoch 7/10
93/93 [==============================] - 651s 7s/step - loss: 2.1482
Epoch 8/10
93/93 [==============================] - 649s 7s/step - loss: 2.0421
Epoch 9/10
93/93 [==============================] - 689s 7s/step - loss: 1.9246
Epoch 10/10
93/93 [==============================] - 649s 7s/step - loss: 1.7944


In [56]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [58]:
generate_text(model, tokenizer, init_sentence="<start>  i love")

'<start> i love flowers deer marble slip entail entail entail entail rheum rheum rheum woodcock ghostly contest flatterer flatterer rheum '

### train_test_split 이부분에서 잘 안돌아갔는데.. 결과는 잘나왔다.
epoch가 10회로 적었지만 loss가  1.7 까지 나와주었다.. 이것은 
embedding_size = 512, hidden_size = 2048 로 늘려서 다행히 결과가 잘 나와주었다. 시간이 많이 잡아먹을수있으므로 적절한 size를 경험적,감각적으로 알수있어야 할듯보인다.